<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#train" data-toc-modified-id="train-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>train</a></span><ul class="toc-item"><li><span><a href="#with-predicted-labels" data-toc-modified-id="with-predicted-labels-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>with predicted labels</a></span></li><li><span><a href="#with-predicted-logits" data-toc-modified-id="with-predicted-logits-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>with predicted logits</a></span></li></ul></li></ul></div>

In [ ]:
%run ./PASTEL_MTL_training_utils.ipynb

In [ ]:
os.environ['WANDB_NOTEBOOK_NAME'] = 'PASTEL train on prediction.ipynb'
import wandb
wandb.init(project="PASTEL train on prediction", entity="fsu-dsc-cil", dir='/scratch/data_jz17d/wandb_tmp/')

wandb: Currently logged in as: cpuyyp (fsu-dsc-cil). Use `wandb login --relogin` to force relogin


# train

## with predicted labels

In [ ]:
p=0.8
label_prefix='predicted_' # 'predicted_' or 'logits_'

my_training_args = MyTrainingArgs(selected_tasks=list(tasks2labels.keys()),
                                  model_name=f'PASTEL train on predicted labels p={p}',
                                  base_model_name='bert-base-uncased',
                                  freeze_bert=False,
                                  use_pooler=False,
                                  num_epoch=5,
#                                   data_limit=30000,
                                 )

hg_training_args = TrainingArguments(
    output_dir=my_training_args.model_folder,   # output directory
    num_train_epochs=my_training_args.num_epoch,     # total number of training epochs
    per_device_train_batch_size=my_training_args.batch_size,  # batch size per device during training
    per_device_eval_batch_size=my_training_args.batch_size,   # batch size for evaluation
    warmup_steps=my_training_args.num_warmup_steps,    # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=f"{my_training_args.model_folder}/logs",  # directory for storing logs
    logging_first_step = True, 
    evaluation_strategy="epoch",     # evaluate each `logging_steps`
    save_total_limit = 1,
    save_strategy = 'epoch',
    report_to = 'wandb',
#     load_best_model_at_end=True, # decide on loss
)

model = init_model(my_training_args)
freeze_model(model, my_training_args.freeze_bert)

train_dataset = MyDataset(my_training_args, f'p={p}_predicted', label_prefix=label_prefix)
val_dataset = MyDataset(my_training_args, 'valid')

trainer = MyTrainer(
    model=model,   # the instantiated Transformers model to be trained
    args=hg_training_args,                  # training arguments, defined above
    tokenizer=model.tokenizer, 
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)
trainer.train()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
***** Running training *****
  Num examples = 31806
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, dist

Epoch,Training Loss,Validation Loss,Accuracy Country,F1 Country,Precision Country,Recall Country,Accuracy Politics,F1 Politics,Precision Politics,Recall Politics,Accuracy Tod,F1 Tod,Precision Tod,Recall Tod,Accuracy Age,F1 Age,Precision Age,Recall Age,Accuracy Education,F1 Education,Precision Education,Recall Education,Accuracy Ethnic,F1 Ethnic,Precision Ethnic,Recall Ethnic,Accuracy Gender,F1 Gender,Precision Gender,Recall Gender
1,2.904800,14.768373,0.975188,0.329146,0.325063,0.333333,0.471429,0.341448,0.314046,0.375249,0.446115,0.206354,0.209450,0.255830,0.420802,0.149393,0.236990,0.163061,0.401253,0.146594,0.168016,0.168759,0.825564,0.189769,0.236170,0.190032,0.724311,0.426309,0.514238,0.428488


***** Running Evaluation *****
  Num examples = 3990
  Batch size = 32
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control 

## with predicted logits

In [ ]:
p=0.8
label_prefix='logits_' # 'predicted_' or 'logits_'

my_training_args = MyTrainingArgs(selected_tasks=list(tasks2labels.keys()),
                                  model_name=f'PASTEL train on predicted logits p={p}',
                                  base_model_name='bert-base-uncased',
                                  freeze_bert=False,
                                  use_pooler=False,
                                  num_epoch=5,
#                                   data_limit=30000,
                                 )

hg_training_args = TrainingArguments(
    output_dir=my_training_args.model_folder,   # output directory
    num_train_epochs=my_training_args.num_epoch,     # total number of training epochs
    per_device_train_batch_size=my_training_args.batch_size,  # batch size per device during training
    per_device_eval_batch_size=my_training_args.batch_size,   # batch size for evaluation
    warmup_steps=my_training_args.num_warmup_steps,    # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=f"{my_training_args.model_folder}/logs",  # directory for storing logs
    logging_first_step = True, 
    evaluation_strategy="epoch",     # evaluate each `logging_steps`
    save_total_limit = 1,
    save_strategy = 'epoch',
    report_to = 'wandb',
#     load_best_model_at_end=True, # decide on loss
)

model = init_model(my_training_args)
freeze_model(model, my_training_args.freeze_bert)

train_dataset = MyDataset(my_training_args, f'p={p}_predicted', label_prefix=label_prefix)
val_dataset = MyDataset(my_training_args, 'valid')

trainer = MyTrainer(
    model=model,   # the instantiated Transformers model to be trained
    args=hg_training_args,                  # training arguments, defined above
    tokenizer=model.tokenizer, 
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)
trainer.train()

In [ ]:
wandb.finish()